# **Análise de clientes, empréstimos e fraudes**

Um escritório contábil, em nome de uma plataforma online de crédito da Alemanha nos contratou para uma consultoria para fazer algumas análises e criar um modelo que identifica possíveis fraudadores. 

Foi disponibilizado uma pequena amostra dos dados dos clientes, porém, heterogênea o suficiente para traçar os perfis, as estratégias as análises e os modelos de detecção de fraude.

## Primeiro passo: preparo dos dados para análises e manipulações:

Essa primeira etapa se faz necessária para identificar as informações dos dados como outliers, estrutura das variáveis (colunas numéricas, datas, textos), informações faltantes, exclusão de informações que não são úteis e demais tratamentos pertinentes.

In [3]:
# Importação dos dados
import pandas as pd
pd.set_option('display.max_columns', 21) # Codigo para mostrar todas as colunas do dataset.
dados = pd.read_csv('german.csv')
dados.sample(5)

,conta,duração,historico,motivo,quantia,poupança,emprego,taxa,status,garantia,residencia,propriedades,idade,financiamentos,moradia,creditos,trabalho,dependentes,telefone,estrangeiro,pagador
367,negativo,18,pagamento em dia,móveis,3650,<100,< 1 ano,1,feminino/divorciado,nenhum,4,carro,22,nenhum,alugada,1,nível 2,1,não,não,bom
468,sem conta,33,já atrasou pagamentos,negócios,2764,<100,"[1,4) anos",2,feminino/divorciado,nenhum,2,carro,26,nenhum,própria,2,nível 2,1,sim,não,bom
639,negativo,42,já atrasou pagamentos,radio/televisão,4370,<100,"[4,7) anos",3,masculino/solteiro,nenhum,2,seguro de vida,26,bancos,própria,2,nível 2,2,sim,não,mau
923,[0-200),12,pagamento em dia,carro novo,2002,<100,"[4,7) anos",3,masculino/solteiro,nenhum,4,seguro de vida,30,nenhum,alugada,1,nível 2,2,sim,não,bom
589,negativo,12,conta crítica,móveis,2246,<100,> 7 anos,3,masculino/solteiro,nenhum,3,seguro de vida,60,nenhum,própria,2,nível 2,1,não,não,mau


In [8]:
# Verificando as informações das colunas das variaveis
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   conta           1000 non-null   object
 1   duração         1000 non-null   int64 
 2   historico       1000 non-null   object
 3   motivo          1000 non-null   object
 4   quantia         1000 non-null   int64 
 5   poupança        1000 non-null   object
 6   emprego         1000 non-null   object
 7   taxa            1000 non-null   int64 
 8   status          1000 non-null   object
 9   garantia        1000 non-null   object
 10  residencia      1000 non-null   int64 
 11  propriedades    1000 non-null   object
 12  idade           1000 non-null   int64 
 13  financiamentos  1000 non-null   object
 14  moradia         1000 non-null   object
 15  creditos        1000 non-null   int64 
 16  trabalho        1000 non-null   object
 17  dependentes     1000 non-null   int64 
 18  telefone 

* Esta base de dados possui apenas mil clientes, porém, contém 21 variáveis ou seja, pode-se extrair diversas informações e insights para o setor de cobrança, marketing, risco entre outros.
* Nenhuma informação faltante ou nula. Isso é raro acontecer numa base de dados do dia a dia, porém, o cliente aqui quis facilitar a nossa vida. Obrigado.

In [5]:
# Verificando as saídas das variáveis
dados.nunique()

conta               4
duração            33
historico           5
motivo             10
quantia           921
poupança            5
emprego             5
taxa                4
status              4
garantia            3
residencia          4
propriedades        4
idade              53
financiamentos      3
moradia             3
creditos            4
trabalho            4
dependentes         2
telefone            2
estrangeiro         2
pagador             2
dtype: int64

Grande parte das variáveis são categóricas (no sentido de que, ao preencher o formulário, o cliente teve disponível opções ao invés de campo livre. Nesse caso facilita muito na análise).

Vamos às legendas das variáveis:
* Conta: qtde de $ disponível em conta corrente
* Duração: qtde de meses que o cliente possui a conta
* Histórico: observações da movimentação da conta
* Motivo: de ter pego o empréstimo
* Quantia: pego emprestada
* Poupança: valor disponivel na conta poupança
* Emprego: tempo no emprego atual
* Taxa: de empréstimo (1 mais alta, 3 mais baixa)
* Status: civil
* Garantia: para o empréstimo
* Residencia: qtde de residencias
* Propriedades: bens declarados ao banco
* Idade
* Financiamentos: que possui aqui ou em outra instituição
* Moradia: própria/alugada
* Creditos: (é um score de 1-4)
* Dependentes: sim ou não 
* Telefone: tem numero cadastrado no banco
* Estrangeiro: se é morador na Alemanha mesmo. (curiosidade: muitos luxemburgueses trabalham e fazem compras na alemanha, logo, também podem pesquisar taxas de empréstimo no outro país)
* Pagador: é a variável alvo do modelo de machine learning. Será utilizada para dados de treino e teste para fazer as futuras previsões.


In [9]:
# Breve análise estatística das variaveis numéricas
dados.describe()

,duração,quantia,taxa,residencia,idade,creditos,dependentes
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.155000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.362086
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000


In [11]:
# Mostra a descrição das categóricas
dados.describe(include=['object']) 

,conta,historico,motivo,poupança,emprego,status,garantia,propriedades,financiamentos,moradia,trabalho,telefone,estrangeiro,pagador
count,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
unique,4,5,10,5,5,4,3,4,3,3,4,2,2,2
top,sem conta,pagamento em dia,radio/televisão,<100,"[1,4) anos",masculino/solteiro,nenhum,carro,nenhum,própria,nível 2,não,não,bom
freq,394,530,280,603,339,548,907,332,814,713,630,596,963,700


Algumas informações iniciais:

- A média de tempo de uma conta é de 21 meses: Ou seja, a maioria dos clientes tem conta cadastrada há menos de dois anos.
- O cliente que está há mais tempo tem seis anos de casa.
- Claramente o nosso cliente é uma fintech nova.
- A média de idade dos clientes é de 36 anos, logo, não é voltada para o público jovem nem para aposentados, apesar de ter clientes de 19 até 75 anos.
- A base de dados não informa a quantidade de dependentes quando o cliente possui, infelizmente.
- O maior valor emprestado foi de 18.424 euros e o menor foi de 250, logo, a empresa não trabalha com PJ, apenas contas PF e de pequenos valores.
- A média da taxa ficou em torno do valor 3, logo, a empresa atrai os clientes com taxas mais baixas.

A maioria dos clientes:

- Não possue dinheiro em conta, logo, os empréstimos foram feitos e sacados.
- Pagam em dia.
- Resolveram pegar dinheiro emprestado para comprar equipamentos eletrônicos. Isso nos dá outra indicação sobre a Fintech. Se ela não pertence á alguma grande rede de varejo.
- Tem menos de 100 euros na conta poupança
- Estão no mesmo emprego há mais de um ano
- São homens, solteiros
- Não deram nada de garantia
- Tem carro
- Não tem outro financiamento na agência
- Tem moradia própria
- Reside no próprio país.


Bem, a base de dados não requer nenhum tratamento prévio, logo, está pronta para as análises das informações e criação de modelos.